<a href="https://colab.research.google.com/github/chanlenium/Android-Mobile-App/blob/main/DataAnalytics/KNN_iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

< KNN(K-Nearest Neighbor) 알고리즘 >
- 서로 가까운 점들은 유사하다는 가정하에, 데이터로부터 거리가 가까운 K개의 다른 데이터의 정답(목표값)을 참조하여 분류
- 변수별 단위가 무엇이냐에 따라 거리가 달라지고, 분류 결과가 달라질 수 있기 때문에 사전에 데이터를 표준화 해야함
- K값에 다라 분류(예측)의 정확도가 달라지므로 적절한 K값을 찾는 것이 매우 중요함
- 일례로, K=3인 경우 새로운 데이터로부터 가장 가까운 이웃 3개를 찾고, 그 중에서 가장 개수가 많은 값으로 분류함

(분석 목표 예) 꽃잎의 길이와 너비, 꽃받침의 길이와 너비를 가지고 붓꽃의 품종(3종류)을 분류(예측)함

In [ ]:
# 분석에 필요한 패키지 임포트
import numpy as np
import pandas as pd
import sklearn

# KNN 분류모델을 위한 패키지
from sklearn.neighbors import KNeighborsClassifier

# 학습/테스트 데이터셋 분리를 위한 패키지
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 불러오기
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
# 데이터프레임의 info()함수를 사용하여 데이터셋에 결측값 존재 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
# 데이터프레임의 기술통계를 보여줌
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
# 데이터 전처리
# 각 독립변수에 대해 Min-Max 정규화 (모든 값을 0~1사이의 값으로 변환)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[["sepal_length"]] = scaler.fit_transform(df[["sepal_length"]])
df[["sepal_width"]] = scaler.fit_transform(df[["sepal_width"]])
df[["petal_length"]] = scaler.fit_transform(df[["petal_length"]])
df[["petal_width"]] = scaler.fit_transform(df[["petal_width"]])
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,0.222222,0.625000,0.067797,0.041667,setosa
1,0.166667,0.416667,0.067797,0.041667,setosa
2,0.111111,0.500000,0.050847,0.041667,setosa
3,0.083333,0.458333,0.084746,0.041667,setosa
4,0.194444,0.666667,0.067797,0.041667,setosa
...,...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667,virginica
146,0.555556,0.208333,0.677966,0.750000,virginica
147,0.611111,0.416667,0.711864,0.791667,virginica
148,0.527778,0.583333,0.745763,0.916667,virginica


In [ ]:
# 분석 데이터셋 준비
# X는 독립변수, y는 종속변수
X = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
y = df["species"]

In [ ]:
# 분석데이터셋 분할(8:2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 11)

In [ ]:
print(X_train.shape)  # 학습 독립변수
print(X_test.shape) # 테스트 독립변수
print(y_train.shape)  # 학습 종속변수
print(y_test.shape) # 테스트 종속변수

(120, 4)
(30, 4)
(120,)
(30,)


In [ ]:
# 데이터 분석 수행
# KNeighborsClassfier 객체 생성
knn = KNeighborsClassifier(n_neighbors = 3)
# 학습 수행
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
# 학습이 완료된 dt 객체에서 테스트 데이터셋으로 예측 수행
pred = knn.predict(X_test)
pred

array(['virginica', 'virginica', 'versicolor', 'versicolor', 'virginica',
       'setosa', 'versicolor', 'setosa', 'setosa', 'versicolor',
       'versicolor', 'versicolor', 'versicolor', 'virginica', 'virginica',
       'setosa', 'virginica', 'versicolor', 'virginica', 'virginica',
       'versicolor', 'setosa', 'setosa', 'versicolor', 'setosa', 'setosa',
       'virginica', 'versicolor', 'setosa', 'versicolor'], dtype=object)

In [ ]:
# 모델 성능 - 정확도 측정
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print(acc)

0.9333333333333333


In [ ]:
# 모델성능평가 - Confusion Matrix 계산
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pred)
print(mat)

[[ 9  0  0]
 [ 0 10  0]
 [ 0  2  9]]


In [ ]:
# 모델성능평가 - 평가지표 계산
from sklearn.metrics import classification_report
rpt = classification_report(y_test, pred)
print(rpt)

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         9
  versicolor       0.83      1.00      0.91        10
   virginica       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.94      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30

